### Loading Dependencies
***

In [65]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
from flask import Flask, render_template, jsonify, redirect
from flask_pymongo import PyMongo
import scrape_hw
import collections

### Getting latest news and titles on Mars
***

In [2]:
url = 'https://mars.nasa.gov/news/'

In [3]:
response = requests.get(url)

In [4]:
soup = bs(response.text, 'html.parser')

In [5]:
#print(soup.prettify)

In [6]:
results_title = soup.find_all('div', class_='content_title')

In [7]:
results_text = soup.find_all('div', class_='rollover_description_inner')

In [8]:
news_title = []
for result in results_title:
    title = result.find('a').text.strip()
    link = result.find('a').get('href')
    news_title.append(title)

In [9]:
news_p = []
for result in results_text:
    text = result.text.strip()
    news_p.append(text)

### Getting the featured image of space or Mars
***

In [11]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
query_url = 'https://www.jpl.nasa.gov/spaceimages/'

In [12]:
browser.visit(query_url)

In [13]:
for x in range(1):
    html = browser.html
    soup = bs(html, 'html.parser')
    image_link = soup.find('article', class_='carousel_item').get('style')
    image_link = image_link[36:]
    image_link = image_link[:-3]
    feature_image_url = query_url + image_link

In [88]:
print(feature_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17462-1920x1200.jpg


In [98]:
scrape_dict = {}
scrape_dict['feature_image'] = feature_image_url

In [109]:
scrape_dict['feature_image']

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17462-1920x1200.jpg'

### Gettingt the latest Twit about Mars
***

In [15]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
query_url = 'https://twitter.com/marswxreport?lang=en'

In [16]:
browser.visit(query_url)

In [17]:
for x in range(1):
    html = browser.html
    soup = bs(html, 'html.parser')
    mars_weather = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

In [18]:
#print(mars_weather)

### Using Pandas DF to export Mars facts HTML to a dictionary
***

In [19]:
url = 'https://space-facts.com/mars/'

In [20]:
tables = pd.read_html(url)

In [21]:
tables[0].columns = ['characteristic', 'value']

In [22]:
mars_df = tables[0]

In [33]:
mars_df

,characteristic,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [34]:
mars_dict = mars_df.to_dict('records')

In [75]:
mars_dict

[{'characteristic': 'Equatorial Diameter:', 'value': '6,792 km'},
 {'characteristic': 'Polar Diameter:', 'value': '6,752 km'},
 {'characteristic': 'Mass:', 'value': '6.42 x 10^23 kg (10.7% Earth)'},
 {'characteristic': 'Moons:', 'value': '2 (Phobos & Deimos)'},
 {'characteristic': 'Orbit Distance:', 'value': '227,943,824 km (1.52 AU)'},
 {'characteristic': 'Orbit Period:', 'value': '687 days (1.9 years)'},
 {'characteristic': 'Surface Temperature:', 'value': '-153 to 20 °C'},
 {'characteristic': 'First Record:', 'value': '2nd millennium BC'},
 {'characteristic': 'Recorded By:', 'value': 'Egyptian astronomers'}]

In [77]:
scrape_dict['facts'] = mars_dict

In [78]:
scrape_dict

{'facts': [{'characteristic': 'Equatorial Diameter:', 'value': '6,792 km'},
  {'characteristic': 'Polar Diameter:', 'value': '6,752 km'},
  {'characteristic': 'Mass:', 'value': '6.42 x 10^23 kg (10.7% Earth)'},
  {'characteristic': 'Moons:', 'value': '2 (Phobos & Deimos)'},
  {'characteristic': 'Orbit Distance:', 'value': '227,943,824 km (1.52 AU)'},
  {'characteristic': 'Orbit Period:', 'value': '687 days (1.9 years)'},
  {'characteristic': 'Surface Temperature:', 'value': '-153 to 20 °C'},
  {'characteristic': 'First Record:', 'value': '2nd millennium BC'},
  {'characteristic': 'Recorded By:', 'value': 'Egyptian astronomers'}]}

In [85]:
scrape_dict['facts'][0]['characteristic']

'Equatorial Diameter:'

### Images of Mars 4 hemispheres
***

In [200]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [201]:
browser.visit(url)

In [215]:
hemisphere_image_urls = []
image_dict = {}

for x in range(1):
    html = browser.html
    soup = bs(html, 'html.parser')
    mars_image_group = soup.find_all('div', class_='collapsible results')
    
#    print(mars_image_group)
    
    for item in mars_image_group:
        image_items = item.findAll('div', class_='item')
#        print(image_items)
 
        counter = 0
        for link in image_items:
            image_link = link.find('a').get('href')
            image_link = image_link[24:]
#            print(image_link)
            image_text = link.find('h3').text
            image_url = 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/' + image_link + '.tif/full.jpg'
#            print(image_text, image_url)
            image_dict = {'title':image_text, 'image_url'[counter]: image_url}
            hemisphere_image_urls.append(image_dict)
            counter = counter + 1

In [217]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'i': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'm': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'a': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'g': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [198]:
hemisphere_image_urls[3]

{'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
 'title': 'Valles Marineris Hemisphere Enhanced'}